In [1]:
import os
import sys
import numpy as np
import pandas as pd
import warnings
from datetime import date, datetime
import statsmodels.api as sm
from IPython.core.display import display, HTML
import snowflake.connector 
import matplotlib.pyplot as plt
import json
from scipy import stats

warnings.filterwarnings('ignore')
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 550)
pd.set_option('display.max_columns', 550)
import snowflake.connector
cnx = snowflake.connector.connect(
    user='nmittleman@toasttab.com',
    account='toast.us-east-1',
    authenticator='externalbrowser'
    )

%matplotlib inline

def QueryDWH(query, cnx, col=[]):
    cur = cnx.cursor()
    cur.execute(query)
    # If there is no col input, then get the columns from the return of the
    # snowflake query
    if not col:
        df = pd.DataFrame(cur.fetchall())
        if not df.empty:
            df.columns = [desc[0] for desc in cur.description]
    else:
        df = pd.DataFrame(cur.fetchall(), columns=col)

    return df
QueryDWH("""USE DATABASE TOAST""", cnx)
QueryDWH("""USE WAREHOUSE TOAST_WH""", cnx)

# 25th Percentile
def q25(x):
    return x.quantile(0.25)

# 50th Percentile
def q50(x):
    return x.quantile(0.5)

# 90th Percentile
def q75(x):
    return x.quantile(0.75)

C:\Users\Toast User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Toast User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Toast User\anaconda3\lib\site-packages\snowflake\connector\options.py:94: UserWarning: You have an incompatible version of 'pyarrow' installed (0.16.0), please install a version that adheres to: 'pyarrow<3.1.0,>=3.0.0; extra == "pandas"'
  warn_incompatible_dep(


# Snowflake Queries

## First Query Gets Customers details from the customer table <br>


In [2]:
query = """
        WITH max_dt AS 
        (SELECT customer_id, MAX(dt) AS dt
        FROM payments.daily_customer_details
        GROUP BY 1)
        SELECT a.customer_id
             , a.region
             , a.restaurant_type
             , a.restaurant_service_model
             , a.pos_first_order_date
             , DATE_TRUNC('Quarter', a.pos_first_order_date)
        FROM payments.daily_customer_details a
        INNER JOIN max_dt b
        ON a.customer_id = b.customer_id
        AND a.dt = b.dt;
        """

cust = QueryDWH(query, cnx, ['Customer Id', 'Region', 'Type', 'Service Model', 'FOD', 'FOQ'])
cust['FOD'] = pd.to_datetime(cust['FOD']).dt.date
cust['FOQ'] = pd.to_datetime(cust['FOQ']).dt.date

## Second Query gets the modules that a customer has for each quarter


In [3]:
query = """
        WITH cust_module AS
            (SELECT  customer_id
                   , DATE_TRUNC('Quarter', dt) AS quarter
                   , SUM(IFF(product_name IN ('Software Monthly Subscription',
                                              'Additional Tablet Monthly Software Subscription',
                                              'Caller ID Monthly Subscription'), saas_quantity, 0)) AS core_count
                   , SUM(IFF(product_name IN ('Enterprise Data Management Monthly Subscription',
                                              'Enterprise Software Monthly Subscription'), saas_quantity, 0)) AS ent_count 
                   , SUM(IFF(product_name IN ('Handheld Monthly Software Subscription'), saas_quantity, 0)) AS hh_count
                   , SUM(IFF(product_name IN ('Online Ordering Monthly Subscription'), saas_quantity, 0)) AS oo_count
                   , SUM(IFF(product_name IN ('Gift Card Program Monthly Subscription'), saas_quantity, 0)) AS gc_count
                   , SUM(IFF(product_name IN ('API Monthly Subscription'), saas_quantity, 0)) AS api_count
                   , SUM(IFF(product_name IN ('Loyalty Program Monthly Subscription'), saas_quantity, 0)) AS loy_count
                   , SUM(IFF(product_name IN ('Marketing Monthly Subscription - 1st Location', 'Marketing Monthly Subscription - Additional Location', 'CRM Monthly Subscription'), saas_quantity, 0)) AS marketing_count
                   , SUM(IFF(product_name IN ('Toast Order & Pay', 'Toast Mobile Order & Pay'), saas_quantity, 0)) AS opt_count
                   , SUM(IFF(product_name IN ('Kitchen Display Screen Monthly Subscription'), saas_quantity, 0)) AS kds_count
                   , SUM(IFF(product_name IN ('Kiosk Monthly Subscription'), saas_quantity, 0)) AS kiosk_count
                   , SUM(IFF(product_name IN ('Inventory Monthly Subscription'), saas_quantity, 0)) AS inv_count
               , SUM(IFF(product_name IN ('xtraCHEF Pro' , 'Invoice Automation', 'Recipe Costing', 'Inventory Management'), saas_quantity, 0)) AS xc_count
               , SUM(IFF(product_name IN ('DoorDash Integration', 'GrubHub Integration', 'UberEats Integration') AND dt >= '2021-04-01', saas_quantity, 0)) AS three_pd_count
               
               ,  IFF(ent_count > 0, 1, 0) + IFF(hh_count > 0, 1, 0) + IFF(oo_count > 0, 1, 0) + IFF(gc_count > 0, 1, 0) 
                + IFF(api_count > 0, 1, 0) + IFF(loy_count > 0, 1, 0) + IFF(marketing_count > 0, 1, 0) + IFF(kds_count > 0, 1, 0) 
                + IFF(opt_count > 0, 1, 0) + IFF(kiosk_count > 0, 1, 0) + IFF(inv_count > 0, 1, 0) + IFF(three_pd_count > 0, 1, 0)
                + IFF(xc_count > 0, 1, 0) AS module_count_live_today
              FROM analytics_core_arr.saas_daily_status
              WHERE LAST_DAY(dt, 'Quarter') = dt
              AND dt >= '2018-01-01'
              AND pos_status = 'Live'
              AND saas_status = 'Live'
              GROUP BY 1,2), tds AS 
              (SELECT DATE_TRUNC('Quarter', dt) AS quarter
                    , customer_id
                    , SUM(TDS_TRANSACTIONS) AS tds_tx
                    , IFF(tds_tx > 0, 1, 0) AS tds_count
                 FROM payments.daily_customer_details 
                 WHERE LAST_DAY(dt, 'Quarter') = LAST_DAY(dt, 'Month')
                 GROUP BY 1,2
               ), ec AS 

              (        SELECT DISTINCT
                        DATE_TRUNC('Quarter', p.check_date) AS quarter
                      , t.customer_id
                      , 1 AS ec
                    FROM source_estratex.payroll_earning_historical_entity_current e
                    LEFT JOIN source_estratex.payroll_historical_entity_current p
                        ON e.estratex_payroll_uuid = p.estratex_payroll_uuid
                    LEFT JOIN source_estratex.location_entity_current l
                        ON e.estratex_location_uuid = l.estratex_location_uuid
                    INNER JOIN analytics_core.toastorders_customer_bridge t
                    ON l.restaurant_guid = t.toastorders_restaurant_guid
                    WHERE DATE_TRUNC(month, p.check_date) >= '2018-01-01'),

              live_count AS
            (SELECT  DATE_TRUNC('Quarter', dt) AS quarter
                   , customer_id
                   , 1 AS live_locs
              FROM payments.daily_customer_details
              WHERE LAST_DAY(dt, 'Quarter') = dt
              AND dt >= '2018-01-01'
              AND pos_status = 'Live')
            (SELECT c.customer_id
                 , c.quarter
                 , DATE_TRUNC('Quarter', DATEADD('Days', -1, c.quarter))
                 , ZEROIFNULL(b.tds_count) AS  tds_count
                 , a.core_count
                 , a.ent_count 
                 , a.hh_count
                 , a.oo_count
                 , a.gc_count
                 , a.api_count
                 , a.loy_count
                 , a.marketing_count
                 , a.opt_count
                 , a.kds_count
                 , a.kiosk_count
                 , a.inv_count
                 , a.three_pd_count
                 , a.xc_count
                 , ZEROIFNULL(d.ec) AS ec_count
                 , a.module_count_live_today + ZEROIFNULL(b.tds_count) + ZEROIFNULL(d.ec) AS unique_mod
            FROM live_count c
            LEFT JOIN cust_module a
            ON a.quarter = c.quarter
            AND a.customer_id = c.customer_id
            LEFT JOIN tds b
            ON a.quarter = b.quarter
            AND a.customer_id = b.customer_id
            LEFT JOIN ec d
            ON a.quarter = d.quarter
            AND a.customer_id = d.customer_id)
        """

modCounts = QueryDWH(query, cnx, ['Customer Id', 'Quarter', 'Prev Quarter', 'TDS','Core', 'Ent', 'HH', 'OO', 'GC', 'API', 'Loy', 'Mkt', 'OPT', 'KDS', 'Kiosk', 'Inv', '3PD', 'XC', 'EC', 'Mod Count'])

for col in ['TDS','Core', 'Ent', 'HH', 'OO', 'GC', 'API', 'Loy', 'Mkt', 'OPT', 'KDS', 'Kiosk', 'Inv', '3PD', 'XC', 'EC', 'Mod Count']:
    modCounts[col] = modCounts[col].astype(float)



In [4]:
# Set data columns to date type
# Create a 'Next Year' column - this is actually 15 months from the quarter start, because what we want 
# is the end of next year. for example. for Q1 - 2021, the Quarter date is 1/1/2021. and one year from the end
# of Q1 - 2021 is the start of Q2 - 2022. hence 15 months
modCounts['Quarter'] = pd.to_datetime(modCounts['Quarter']).dt.date
modCounts['Prev Quarter'] = pd.to_datetime(modCounts['Prev Quarter']).dt.date
modCounts['Next Year'] = modCounts['Quarter'] + pd.DateOffset(months=15)
modCounts['Next Year'] = pd.to_datetime(modCounts['Next Year']).dt.date

In [5]:
# Create a dataframe that has the modules for each quarter as well as the modules at the start of 
# the one year prior
modCountsLastYear = modCounts.copy()
for col in ['TDS','Core', 'Ent', 'HH', 'OO', 'GC', 'API', 'Loy', 'Mkt', 'OPT', 'KDS', 'Kiosk', 'Inv', '3PD', 'XC', 'EC', 'Mod Count', 'Loans']:
    modCountsLastYear.rename(columns = {col: f'{col} Last Year'}, inplace=True)
    
yoyModCounts = modCounts[['Customer Id', 'Quarter', 'Prev Quarter']].\
                    merge(modCounts.drop(['Prev Quarter', 'Next Year', 'Next Quarter'], axis=1, errors='ignore').rename(columns = {'Quarter': 'Quarter'}) ).\
                    merge(modCountsLastYear.drop(['Prev Quarter', 'Quarter', 'Next Quarter'], axis=1, errors='ignore').rename(columns = {'Next Year': 'Quarter'} ))
yoyModCounts['Mod Count Change'] = yoyModCounts['Mod Count'] - yoyModCounts['Mod Count Last Year']

## Third Query gets the GMV and GMV per Day for each customer for each quarter

In [6]:
query = """
            SELECT CUSTOMER_ID
                , DATE_TRUNC('Quarter', dt) AS quarter
                , SUM(gmv) AS gmv
                , COUNT(DISTINCT IFF(gmv > 0, dt, NULL)) AS active_days               
                , MIN(IFF(gmv > 0, gmv, NULL)) AS min_gmv_per_day
                , APPROX_PERCENTILE(IFF(gmv > 0, gmv, NULL), 0.25 ) AS gmv_25th
                , MEDIAN(IFF(gmv > 0, gmv, NULL)) AS median_gmv_per_day            
                , APPROX_PERCENTILE(IFF(gmv > 0, gmv, NULL), 0.75 ) AS gmv_75th
                , MAX(IFF(gmv > 0, gmv, NULL)) AS max_gmv_per_day
            FROM PAYMENTS.daily_customer_details
            WHERE pos_status = 'Live'
            AND pos_first_order_date < DATE_TRUNC('Quarter' ,dt)
            GROUP BY 1, 2;
        """
quarterGMV = QueryDWH(query, cnx, ['Customer Id', 'Quarter', 'GMV', 'Active Days', 'Min GMV PD', '25th Pct GMV PD', 'Med GMV PD', '75th Pct GMV PD', 'Max GMV PD'])
for col in ['GMV', 'Active Days', 'Min GMV PD', '25th Pct GMV PD', 'Med GMV PD', '75th Pct GMV PD', 'Max GMV PD']:
    quarterGMV[col] = quarterGMV[col].astype(float)
    

# Here we actually want to use 12 months from the quarter because we are comparing all GMV in one quarter
# to all GMV in the same quarter the following year
quarterGMV['Quarter'] = pd.to_datetime(quarterGMV['Quarter']).dt.date
quarterGMV['Next Year'] = quarterGMV['Quarter'] + pd.DateOffset(months=12)
quarterGMV['Next Year'] = pd.to_datetime(quarterGMV['Next Year']).dt.date

# Combine GMV, Module, and Customer Data


In [7]:
# Join the quarter GMV to itself to get the GMV from Prev Year
yoyGMV = quarterGMV.merge(quarterGMV[['Customer Id', 'Next Year', 'GMV', 'Active Days',  'Med GMV PD']].\
                          rename(columns = {'Next Year': 'Quarter', 
                                            'GMV': 'GMV Last Year', 
                                            'Active Days': 'Active Days Last Year',
                                            'Med GMV PD': 'Med GMV PD Last Year'}) ).\
drop(['Next Year', 'Min GMV PD', '25th Pct GMV PD', '75th Pct GMV PD', 'Max GMV PD'], axis=1, errors='ignore')

# We define the GMV per Day as the average GMV on a given day
yoyGMV['GMV Per Day'] = yoyGMV['GMV'] / yoyGMV['Active Days']
yoyGMV['GMV Per Day Last Year'] = yoyGMV['GMV Last Year'] / yoyGMV['Active Days Last Year']

# We define growth rate in two ways
# Here GMV Growth is how much total GMV grew Year over Year
yoyGMV['GMV Growth'] = yoyGMV['GMV'] - yoyGMV['GMV Last Year']
yoyGMV['GMV Growth Percent'] = np.round(100 * yoyGMV['GMV Growth'] / yoyGMV['GMV Last Year'], 2)

# Here GMV Per Day Growth is how much average GMV Per Day grew
yoyGMV['GMV Per Day Growth'] = yoyGMV['GMV Per Day'] - yoyGMV['GMV Per Day Last Year']
yoyGMV['GMV Per Day Growth Percent'] = np.round(100 * yoyGMV['GMV Per Day Growth'] / yoyGMV['GMV Per Day Last Year'], 2)

# Here Median GMV Per Day Growth is how much average GMV Per Day grew
yoyGMV['Med GMV Per Day Growth'] = yoyGMV['Med GMV PD'] - yoyGMV['Med GMV PD Last Year']
yoyGMV['Med GMV Per Day Growth Percent'] = np.round(100 * yoyGMV['Med GMV Per Day Growth'] / yoyGMV['Med GMV PD Last Year'], 2)


In [8]:
# Join the GMV and Module information to create a dataset that has Year over Year GMV growth and the modules that 
# each customer has in the current quarter and quarter one year previous
yoyCompare = yoyGMV.merge(yoyModCounts)
yoyCompare['Guest Count'] = yoyCompare[['OO', 'GC', 'Loy', 'Mkt']].sum(axis=1)
yoyCompare['Guest Count Last Year'] = yoyCompare[['OO Last Year', 'GC Last Year', 'Loy Last Year', 'Mkt Last Year']].sum(axis=1)
for mod in ['TDS','Core', 'Ent', 'HH', 'OO', 'GC', 'API', 'Loy', 'Mkt', 'OPT', 'KDS', 'Kiosk', 'Inv', '3PD', 'XC', 'EC', 'Mod Count', 'Guest Count']:
    yoyCompare[f'{mod} Change'] = (yoyCompare[mod] - yoyCompare[f'{mod} Last Year']).fillna(0)


# Get the Year over Year Growth for Customers that ADDED modules

In [9]:
for mod in ['TDS','Core', 'Ent', 'HH', 'OO', 'GC', 'API', 'Loy', 'Mkt', 'OPT', 'KDS', 'Kiosk', 'Inv', '3PD', 'XC', 'EC', 'Mod Count', 'Guest Count']:

    comp = yoyCompare[yoyCompare['GMV Per Day Growth Percent'] > -50]
    comp = comp[comp['GMV Per Day Growth Percent'] < 200]
    
    comp[f'Mod Groups'] = 'No Change'
    comp[f'Mod Groups'] = np.where(comp[f'{mod} Change'] > 0, 'Added', comp[f'Mod Groups'])
    comp[f'Mod Groups'] = np.where(comp[f'{mod} Change'] <= 0, 'Did Not Add', comp[f'Mod Groups'])


    results = pd.DataFrame()
    for quarter in comp.sort_values('Quarter')['Quarter'].unique().tolist():
        tmp = comp[comp['Quarter'] == quarter]

        agg = tmp.groupby(['Mod Groups']).agg({'Customer Id': 'nunique','GMV Per Day Growth Percent': q50, 'GMV': 'sum', 'GMV Last Year': 'sum'})
        agg['Avg'] = np.round(100 * (agg['GMV'] - agg['GMV Last Year']) / agg['GMV Last Year'], 2)
        if len(agg) == 2:
            pVal = stats.ttest_ind(tmp[tmp['Mod Groups'] == 'Added']['GMV Per Day Growth Percent'].to_numpy(), tmp[tmp['Mod Groups'] == 'Did Not Add']['GMV Per Day Growth Percent'].to_numpy())[1]
            pVal = np.round(pVal, 4)
            tmpResults = pd.DataFrame({'# Customers Added Modules': [agg.loc['Added']['Customer Id']],
                                       '# Customers Did Not Add Modules': [agg.loc['Did Not Add']['Customer Id']],
                                       'Added - Typical Growth ': [agg.loc['Added']['GMV Per Day Growth Percent']],
                                       'Did Not Add - Typical Growth': [agg.loc['Did Not Add']['GMV Per Day Growth Percent']],
                                       'Typical Growth Difference': [agg.loc['Added']['GMV Per Day Growth Percent'] - agg.loc['Did Not Add']['GMV Per Day Growth Percent']],
                                       'Growth Rate Difference': [np.round(100 * (agg.loc['Added']['GMV Per Day Growth Percent'] - agg.loc['Did Not Add']['GMV Per Day Growth Percent']) / (agg.loc['Did Not Add']['GMV Per Day Growth Percent']), 2)],

                                       'pVal': [pVal]}, index=[quarter])
            results = pd.concat([results, tmpResults])
    print(mod)
    display(results)
    results.to_csv(f'./Spark 2022 Files/{mod} Added.csv')

TDS


,# Customers Added Modules,# Customers Did Not Add Modules,Added - Typical Growth,Did Not Add - Typical Growth,Typical Growth Difference,Growth Rate Difference,pVal
2020-01-01,1.0,11814.0,-15.260,-6.440,-8.820,136.96,NaN
2020-04-01,239.0,8250.0,-30.270,-20.190,-10.080,49.93,0.0000
2020-07-01,591.0,13016.0,-16.910,-7.530,-9.380,124.57,0.0000
2020-10-01,990.0,14891.0,-13.490,-7.750,-5.740,74.06,0.0000
2021-01-01,1652.0,17727.0,2.500,6.130,-3.630,-59.22,0.0004
2021-04-01,1665.0,18708.0,63.820,56.100,7.720,13.76,0.0000
2021-07-01,1491.0,24486.0,36.450,29.780,6.670,22.40,0.0001
2021-10-01,1527.0,28032.0,36.150,32.525,3.625,11.15,0.0174
2022-01-01,1336.0,30985.0,22.095,19.590,2.505,12.79,0.0178
2022-04-01,1023.0,35330.0,11.160,7.805,3.355,42.99,0.0000


Core


,# Customers Added Modules,# Customers Did Not Add Modules,Added - Typical Growth,Did Not Add - Typical Growth,Typical Growth Difference,Growth Rate Difference,pVal
2019-04-01,462.0,6221.0,5.755,2.030,3.725,183.50,0.0000
2019-07-01,355.0,8190.0,6.540,1.400,5.140,367.14,0.0000
2019-10-01,382.0,9773.0,7.300,2.210,5.090,230.32,0.0000
2020-01-01,481.0,11334.0,-3.270,-6.540,3.270,-50.00,0.0000
2020-04-01,439.0,8050.0,-18.910,-20.535,1.625,-7.91,0.0482
2020-07-01,627.0,12980.0,-0.050,-8.360,8.310,-99.40,0.0000
2020-10-01,733.0,15148.0,-1.460,-8.560,7.100,-82.94,0.0000
2021-01-01,900.0,18479.0,17.735,5.240,12.495,238.45,0.0000
2021-04-01,869.0,19504.0,69.530,56.135,13.395,23.86,0.0000
2021-07-01,1394.0,24583.0,39.250,29.680,9.570,32.24,0.0000


Ent


...

,# Customers Added Modules,# Customers Did Not Add Modules,Added - Typical Growth,Did Not Add - Typical Growth,Typical Growth Difference,Growth Rate Difference,pVal
2019-04-01,136.0,6547.0,3.915,2.27,1.645,72.47,0.8793
2019-07-01,181.0,8364.0,2.520,1.64,0.880,53.66,0.7082
2019-10-01,348.0,9807.0,2.110,2.37,-0.260,-10.97,0.6521
2020-01-01,478.0,11337.0,-7.405,-6.39,-1.015,15.88,0.3899
2020-04-01,350.0,8139.0,-23.730,-20.31,-3.420,16.84,0.1025
2020-07-01,513.0,13094.0,-9.520,-7.93,-1.590,20.05,0.1361
2020-10-01,555.0,15326.0,-9.720,-8.13,-1.590,19.56,0.8756
2021-01-01,696.0,18683.0,5.640,5.74,-0.100,-1.74,0.8038
2021-04-01,564.0,19809.0,61.145,56.76,4.385,7.73,0.2996
2021-07-01,822.0,25155.0,38.450,29.92,8.530,28.51,0.0000


Mod Count


,# Customers Added Modules,# Customers Did Not Add Modules,Added - Typical Growth,Did Not Add - Typical Growth,Typical Growth Difference,Growth Rate Difference,pVal
2019-04-01,1492.0,5191.0,3.890,1.88,2.010,106.91,0.0000
2019-07-01,1942.0,6603.0,3.140,1.12,2.020,180.36,0.0000
2019-10-01,2432.0,7723.0,3.925,1.81,2.115,116.85,0.0000
2020-01-01,3532.0,8283.0,-6.295,-6.52,0.225,-3.45,0.0439
2020-04-01,3090.0,5399.0,-22.545,-19.16,-3.385,17.67,0.0003
2020-07-01,5136.0,8471.0,-10.525,-6.38,-4.145,64.97,0.0000
2020-10-01,6435.0,9446.0,-10.080,-6.74,-3.340,49.55,0.0040
2021-01-01,8183.0,11196.0,4.740,6.46,-1.720,-26.63,0.1602
2021-04-01,7922.0,12451.0,63.525,52.97,10.555,19.93,0.0000
2021-07-01,9332.0,16645.0,34.205,28.41,5.795,20.40,0.0000


Guest Count


,# Customers Added Modules,# Customers Did Not Add Modules,Added - Typical Growth,Did Not Add - Typical Growth,Typical Growth Difference,Growth Rate Difference,pVal
2019-04-01,509.0,6174.0,2.690,2.250,0.440,19.56,0.0004
2019-07-01,667.0,7878.0,2.780,1.510,1.270,84.11,0.0001
2019-10-01,905.0,9250.0,4.640,2.165,2.475,114.32,0.0000
2020-01-01,1873.0,9942.0,-6.310,-6.460,0.150,-2.32,0.0812
2020-04-01,1851.0,6638.0,-24.160,-19.550,-4.610,23.58,0.0004
2020-07-01,3289.0,10318.0,-11.580,-6.905,-4.675,67.70,0.0000
2020-10-01,4490.0,11391.0,-10.585,-7.050,-3.535,50.14,0.0001
2021-01-01,5625.0,13754.0,3.680,6.730,-3.050,-45.32,0.0000
2021-04-01,4310.0,16063.0,66.225,54.550,11.675,21.40,0.0000
2021-07-01,4065.0,21912.0,34.200,29.570,4.630,15.66,0.0000


# See Year over Year GVM Growth by Customers with Growth Package


In [10]:
# Here we look at customers that had components of the growth package
# OO, GC, Mkt, Loy, and OPT for fun!
# We then look at customers who had different groupings of those modules over the course of the year
# ie had OO and Loy at the start of a 12 month period and at the end
# and see if their growth rates compare to customers without any Growth Package modules
results = pd.DataFrame()
for quarter in comp[comp['Quarter'].isin([date(2022,4,1)]) ].sort_values('Quarter')['Quarter'].unique().tolist():

    comp = cust.merge(yoyCompare[yoyCompare['GMV Per Day Growth Percent'] > -50])
    comp = comp[comp['GMV Per Day Growth Percent'] < 200]

    tmp = comp[comp['Quarter'] == quarter]
    tmp['Mod Groups'] = 'Didnt Have'
    tmp['Mod Groups'] = np.where((tmp[['OO', 'GC', 'Loy', 'Mkt']].sum(axis=1) > 0), 'Had', tmp['Mod Groups'])
    
    tmp['Mod Tmp'] = ''
    tmp['Mod Tmp'] = np.where((tmp['OO'] > 0) & (tmp['OO Last Year'] > 0), tmp['Mod Tmp'] + ' OO', tmp['Mod Tmp'])
    tmp['Mod Tmp'] = np.where((tmp['GC'] > 0) & (tmp['GC Last Year'] > 0), tmp['Mod Tmp'] + ' GC', tmp['Mod Tmp'])
    tmp['Mod Tmp'] = np.where((tmp['Loy'] > 0) & (tmp['Loy Last Year'] > 0), tmp['Mod Tmp'] + ' Loy', tmp['Mod Tmp'])
    tmp['Mod Tmp'] = np.where((tmp['Mkt'] > 0) & (tmp['Mkt Last Year'] > 0), tmp['Mod Tmp'] + ' Mkt', tmp['Mod Tmp'])

    
    agg = tmp.groupby(['Mod Tmp']).agg({'Customer Id': 'nunique','GMV Per Day': 'mean','GMV Per Day Last Year': 'mean','GMV Per Day Growth Percent': q50})
    agg['Avg'] = np.round(100 * (agg['GMV Per Day'] - agg['GMV Per Day Last Year']) / agg['GMV Per Day Last Year'], 2)
    agg.sort_values('Customer Id', ascending=False, inplace=True)
    listTmp = agg[agg['Customer Id'] > 500].index.tolist()
    
    for mod in listTmp + ['All']:
        if mod == 'All':
            newTmp = tmp.copy()
        else:
            newTmp = tmp[tmp['Mod Tmp'].isin([mod, ''])]
        agg = newTmp.groupby(['Mod Groups']).agg({'Customer Id': 'nunique','GMV Per Day': 'mean','GMV Per Day Last Year': 'mean','GMV Per Day Growth Percent': q50})
        agg['Avg'] = np.round(100 * (agg['GMV Per Day'] - agg['GMV Per Day Last Year']) / agg['GMV Per Day Last Year'], 2)

        if len(agg) == 2:
            pVal = stats.ttest_ind(newTmp[newTmp['Mod Groups'] == 'Didnt Have']['GMV Per Day Growth Percent'].to_numpy(), newTmp[newTmp['Mod Groups'] == 'Had']['GMV Per Day Growth Percent'].to_numpy())[1]
            pVal = np.round(pVal, 4)
            
            index = pd.MultiIndex.from_tuples([(quarter, mod)], names=["Quarter", "Module"])
            tmpResults = pd.DataFrame({'# Customers Had Modules': [agg.loc['Had']['Customer Id']],
                                       '# Customers Did Not Have Modules': [agg.loc['Didnt Have']['Customer Id']],
                                       'Had - Typical Growth ': [agg.loc['Had']['GMV Per Day Growth Percent']],
                                       'Did Not Have - Typical Growth': [agg.loc['Didnt Have']['GMV Per Day Growth Percent']],
                                       'Typical Growth Difference': [agg.loc['Had']['GMV Per Day Growth Percent'] - agg.loc['Didnt Have']['GMV Per Day Growth Percent']],
                                       'Growth Rate Difference': [np.round(100 * (agg.loc['Had']['GMV Per Day Growth Percent'] - agg.loc['Didnt Have']['GMV Per Day Growth Percent']) / (agg.loc['Didnt Have']['GMV Per Day Growth Percent']), 2)],
                                       'pVal': [pVal]}, index=index)
            results = pd.concat([results, tmpResults])
display(results.sort_values(['Module', 'Quarter']))
results.to_csv(f'./Spark 2022 Files/Growth Package Had.csv')

# Customers Had Modules  \
Quarter    Module                                    
2022-04-01                                   524.0   
            GC                              4501.0   
            GC Loy                          2123.0   
            OO                              4347.0   
            OO GC                           8406.0   
            OO GC Loy                       9401.0   
            OO GC Loy Mkt                   1467.0   
            OO Loy                          1603.0   
           All                             29820.0   

                           # Customers Did Not Have Modules  \
Quarter    Module                                             
2022-04-01                                           6496.0   
            GC                                       6510.0   
            GC Loy                                   6496.0   
            OO                                       6509.0   
            OO GC                                    6497.0   
            OO GC Loy                                6496.0   
            OO GC Loy Mkt                            6496.0   
            OO Loy                                   6496.0   
           All                                       6533.0   

                           Had - Typical Growth   \
Quarter    Module                                  
2022-04-01                                11.895   
            GC                             8.530   
            GC Loy                         7.170   
            OO                             8.560   
            OO GC                          9.560   
            OO GC Loy                      7.870   
            OO GC Loy Mkt                  9.600   
            OO Loy                         9.180   
           All                             8.350   

                           Did Not Have - Typical Growth  \
Quarter    Module                                          
2022-04-01                                         5.395   
            GC                                     5.430   
            GC Loy                                 5.395   
            OO                                     5.430   
            OO GC                                  5.380   
            OO GC Loy                              5.395   
            OO GC Loy Mkt                          5.395   
            OO Loy                                 5.395   
           All                                     5.460   

                           Typical Growth Difference  Growth Rate Difference  \
Quarter    Module                                                              
2022-04-01                                     6.500                  120.48   
            GC                                 3.100                   57.09   
            GC Loy                             1.775                   32.90   
            OO                                 3.130                   57.64   
            OO GC                              4.180                   77.70   
            OO GC Loy                          2.475                   45.88   
            OO GC Loy Mkt                      4.205                   77.94   
            OO Loy                             3.785                   70.16   
           All                                 2.890                   52.93   

                             pVal  
Quarter    Module                  
2022-04-01                 0.0000  
            GC             0.0000  
            GC Loy         0.0078  
            OO             0.0000  
            OO GC          0.0000  
            OO GC Loy      0.1405  
            OO GC Loy Mkt  0.0000  
            OO Loy         0.0000  
           All             0.0000

In [11]:
# Here we look at customers that had cOPT
# We then look at customers who had different groupings of those modules over the course of the year
# ie had OO and Loy at the start of a 12 month period and at the end
# and see if their growth rates compare to customers without any Growth Package modules
results = pd.DataFrame()
for quarter in comp[comp['Quarter'].isin([date(2022,4,1)]) ].sort_values('Quarter')['Quarter'].unique().tolist():

    comp = cust.merge(yoyCompare[yoyCompare['GMV Per Day Growth Percent'] > -50])
    comp = comp[comp['GMV Per Day Growth Percent'] < 200]

    tmp = comp[comp['Quarter'] == quarter]
    tmp['Mod Groups'] = 'Didnt Have'
    tmp['Mod Groups'] = np.where((tmp[['OPT']].sum(axis=1) > 0), 'Had', tmp['Mod Groups'])
    
    tmp['Mod Tmp'] = ''
    tmp['Mod Tmp'] = np.where((tmp['OPT'] > 0) & (tmp['OPT Last Year'] > 0), tmp['Mod Tmp'] + ' OPT', tmp['Mod Tmp'])

    
    agg = tmp.groupby(['Mod Tmp']).agg({'Customer Id': 'nunique','GMV Per Day': 'mean','GMV Per Day Last Year': 'mean','GMV Per Day Growth Percent': q50})
    agg['Avg'] = np.round(100 * (agg['GMV Per Day'] - agg['GMV Per Day Last Year']) / agg['GMV Per Day Last Year'], 2)
    agg.sort_values('Customer Id', ascending=False, inplace=True)
    listTmp = agg[agg['Customer Id'] > 500].index.tolist()
    
    for mod in listTmp + ['All']:
        if mod == 'All':
            newTmp = tmp.copy()
        else:
            newTmp = tmp[tmp['Mod Tmp'].isin([mod, ''])]
        agg = newTmp.groupby(['Mod Groups']).agg({'Customer Id': 'nunique','GMV Per Day': 'mean','GMV Per Day Last Year': 'mean','GMV Per Day Growth Percent': q50})
        agg['Avg'] = np.round(100 * (agg['GMV Per Day'] - agg['GMV Per Day Last Year']) / agg['GMV Per Day Last Year'], 2)

        if len(agg) == 2:
            pVal = stats.ttest_ind(newTmp[newTmp['Mod Groups'] == 'Didnt Have']['GMV Per Day Growth Percent'].to_numpy(), newTmp[newTmp['Mod Groups'] == 'Had']['GMV Per Day Growth Percent'].to_numpy())[1]
            pVal = np.round(pVal, 4)
            
            index = pd.MultiIndex.from_tuples([(quarter, mod)], names=["Quarter", "Module"])
            tmpResults = pd.DataFrame({'# Customers Had Modules': [agg.loc['Had']['Customer Id']],
                                       '# Customers Did Not Have Modules': [agg.loc['Didnt Have']['Customer Id']],
                                       'Had - Typical Growth ': [agg.loc['Had']['GMV Per Day Growth Percent']],
                                       'Did Not Have - Typical Growth': [agg.loc['Didnt Have']['GMV Per Day Growth Percent']],
                                       'Typical Growth Difference': [agg.loc['Had']['GMV Per Day Growth Percent'] - agg.loc['Didnt Have']['GMV Per Day Growth Percent']],
                                       'Growth Rate Difference': [np.round(100 * (agg.loc['Had']['GMV Per Day Growth Percent'] - agg.loc['Didnt Have']['GMV Per Day Growth Percent']) / (agg.loc['Didnt Have']['GMV Per Day Growth Percent']), 2)],
                                       'pVal': [pVal]}, index=index)
            results = pd.concat([results, tmpResults])
display(results.sort_values(['Module', 'Quarter']))
results.to_csv(f'./Spark 2022 Files/OPT Had.csv')

# Customers Had Modules  # Customers Did Not Have Modules  \
Quarter    Module                                                              
2022-04-01                          1290.0                           33533.0   
            OPT                     2820.0                           33533.0   
           All                      2820.0                           33533.0   

                   Had - Typical Growth   Did Not Have - Typical Growth  \
Quarter    Module                                                         
2022-04-01                         9.240                           7.71   
            OPT                   10.755                           7.71   
           All                    10.755                           7.71   

                   Typical Growth Difference  Growth Rate Difference  pVal  
Quarter    Module                                                           
2022-04-01                             1.530                   19.84   0.0  
            OPT                        3.045                   39.49   0.0  
           All                         3.045                   39.49   0.0